In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [3]:
!pip install nlp-synt-data
!rm -rf content/unimi-language-detection/
!git clone https://github.com/tommasoromano/unimi-language-detection.git
import sys
sys.path.insert(0,'/content/unimi-language-detection')
from nlp_synt_data import *
from data.prompts import *
from data.texts import *
from sc import *

Cloning into 'unimi-language-detection'...
remote: Enumerating objects: 481, done.
remote: Counting objects: 100% (481/481), done.
remote: Compressing objects: 100% (349/349), done.
remote: Total 481 (delta 275), reused 334 (delta 128), pack-reused 0 (from 0)
Receiving objects: 100% (481/481), 24.72 MiB | 10.90 MiB/s, done.
Resolving deltas: 100% (275/275), done.


In [4]:
IS_EVAL = True
IS_FINETUNED = True
IS_SEED = False
ORIGINAL_MODEL_NICK = "phi3"
#ORIGINAL_MODEL_NICK = "mistralnemo"
ORIGINAL_MODEL_NAME = "unsloth/Phi-3-mini-4k-instruct"
#ORIGINAL_MODEL_NAME = "unsloth/Mistral-Nemo-Base-2407-bnb-4bit"
VERSION = 'all-v0'
VERSION = 'split-v0'
HF_DS = "romabob/unimi-job"
FILTER_SPLIT = "train" if "split" in VERSION else "all"
FILTER_SPLIT = ""
CSV = sc_csv_name(ORIGINAL_MODEL_NICK, IS_FINETUNED, IS_SEED)
NEW_MODEL = f'{ORIGINAL_MODEL_NICK}-unimi-jobs-{VERSION}'

In [5]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = ORIGINAL_MODEL_NAME, # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

# Unsloth 2024.8: Fast Mistral patching. Transformers = 4.44.0.
# GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
# Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
# Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.34k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

# Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.

Unsloth 2024.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Data Prep

In [7]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset

def get_df(split):
  #dataset = load_dataset("philschmid/guanaco-sharegpt-style", split = "train")
  dataset_raw = load_dataset(HF_DS, split = split)
  dataset_raw = dataset_raw.map(formatting_prompts_func, batched = True,)
  if FILTER_SPLIT != "":
    dataset = dataset_raw.filter(lambda x: x['split'] == FILTER_SPLIT)
  else:
    dataset = dataset_raw
  return dataset

In [8]:
dataset = get_df("train")
dataset

README.md:   0%|          | 0.00/381 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/541k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9425 [00:00<?, ? examples/s]

Map:   0%|          | 0/9425 [00:00<?, ? examples/s]

Dataset({
    features: ['act', 'conversations', 'text'],
    num_rows: 9425
})

In [9]:
dataset[5]["conversations"]

[{'from': 'human',
  'value': 'Sei un assistente che legge ed analizza delle frasi di annunci di lavoro in italiano. Il tuo obbiettivo è analizzare ed identificare se il testo contiene nomi di professioni e se la frase si sta riferendo ad entrambi i generi. Rispondi solamente con il label "NON INCLUSIVO" o "INCLUSIVO".\nStudio Legale Rossi & Associati cerca estetista Legale'},
 {'from': 'gpt',
  'value': 'la frase contiene la parola "estetista" che è di genere "neutro" e quindi la risposta è "INCLUSIVO"'}]

In [10]:
print(dataset[5]["text"])

<|user|>
Sei un assistente che legge ed analizza delle frasi di annunci di lavoro in italiano. Il tuo obbiettivo è analizzare ed identificare se il testo contiene nomi di professioni e se la frase si sta riferendo ad entrambi i generi. Rispondi solamente con il label "NON INCLUSIVO" o "INCLUSIVO".
Studio Legale Rossi & Associati cerca estetista Legale<|end|>
<|assistant|>
la frase contiene la parola "estetista" che è di genere "neutro" e quindi la risposta è "INCLUSIVO"<|end|>



### MINE EVALUATE
### MINE EVALUATE
### MINE EVALUATE
### MINE EVALUATE

In [11]:
if IS_EVAL:

  from unsloth import FastLanguageModel

  _model_name = "romabob/" + NEW_MODEL if IS_FINETUNED else ORIGINAL_MODEL_NAME

  model, _ = FastLanguageModel.from_pretrained(
      model_name = _model_name,
      max_seq_length = max_seq_length,
      dtype = dtype,
      load_in_4bit = load_in_4bit,
  )
  FastLanguageModel.for_inference(model) # Enable native 2x faster inference

#==((====))==  Unsloth 2024.8: Fast Mistral patching. Transformers = 4.44.1.
#   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
#O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
#\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
# "-____-"     Free Apache license: http://github.com/unslothai/unsloth

==((====))==  Unsloth 2024.9: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


adapter_model.safetensors:   0%|          | 0.00/120M [00:00<?, ?B/s]

In [ ]:
if IS_EVAL:
  def TEXT_SEED_v0():
    # /content/unimi-language-detection/
    df = pd.read_csv('/content/unimi-language-detection/data/job_description_seed_dataset_improved_context.csv')
    _map = {
        'YES': 'INCLUSIVO',
        'NO': 'NON INCLUSIVO'
    }
    df['label'] = df.apply(lambda x: _map[x['inclusive phrasing']], axis=1)
    return df[['text','label']].values.tolist()
  # dataset_test = dataset = get_df("test")

  prompts = PromptGenerator.generate(
      PROMPTS_JOB_V0,
      #[["zsl"],],
      PROMPTS_JOB_V0_KEYS,
      )
  data = DataGenerator.generate(
      TEXT_JOB_TEST_v0,
      #TEXT_SEED_v0(),
      #TEXT_JOB_TEST_v0,
      SUBS_JOBS_V0,
      )
  if IS_SEED:
    data = DataGenerator.generate(
        TEXT_SEED_v0(),
        SUBS_JOBS_V0)
  else:
    data = sc_make_test_data()
  # model = 'llama3:instruct'
  n_pass = 1

  def generation_func(prompt, text):

    val = prompt + "\n" + text
    messages = [
        {"from": "human", "value": val},
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for gene ration
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True)
    res = tokenizer.batch_decode(outputs)
    return res[0].split("<|assistant|>")[-1].split("<|end|>")[0]

  _drive = "/content/drive/My Drive/Colab Notebooks/data/"
  ResponseGenerator.generate(f"{_drive}{CSV}", data, prompts,
                              lambda prompt, text: generation_func(prompt, text), n_pass=n_pass)

Loaded 7800 rows.


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Processed 50 rows. Time: 0:02:27.760260, Pace: 2.94 sec/row.
Processed 100 rows. Time: 0:04:37.055961, Pace: 2.77 sec/row.
Processed 150 rows. Time: 0:06:46.663747, Pace: 2.71 sec/row.
Processed 200 rows. Time: 0:09:09.213834, Pace: 2.75 sec/row.
Processed 250 rows. Time: 0:11:20.075031, Pace: 2.72 sec/row.
Processed 300 rows. Time: 0:13:29.766183, Pace: 2.70 sec/row.
Processed 350 rows. Time: 0:15:43.994051, Pace: 2.69 sec/row.
Processed 400 rows. Time: 0:18:02.555106, Pace: 2.71 sec/row.
Processed 450 rows. Time: 0:20:21.479700, Pace: 2.71 sec/row.
Processed 500 rows. Time: 0:22:26.491285, Pace: 2.69 sec/row.
Processed 550 rows. Time: 0:24:31.181056, Pace: 2.67 sec/row.
Processed 600 rows. Time: 0:26:38.073179, Pace: 2.66 sec/row.
Processed 650 rows. Time: 0:28:46.591211, Pace: 2.66 sec/row.
Processed 700 rows. Time: 0:30:54.415645, Pace: 2.65 sec/row.
Processed 750 rows. Time: 0:33:03.724099, Pace: 2.64 sec/row.
Processed 800 rows. Time: 0:35:16.526038, Pace: 2.65 sec/row.
Processed

In [ ]:
if IS_EVAL:
  assert 1 == 2

### Train the model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60 * 6,
        #num_train_epochs=1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        save_strategy = "steps",
        save_steps = 50,
    ),
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

# GPU = Tesla T4. Max memory = 14.748 GB.
# 2.283 GB of memory reserved.

In [ ]:
trainer_stats = trainer.train()

#==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
#   \\   /|    Num examples = 5,712 | Num Epochs = 1
#O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
#\        /    Total batch size = 8 | Total steps = 360
# "-____-"     Number of trainable parameters = 29,884,416

# 0.28 it/s
# duration: 20.59 min

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

### Saving finetuned models

In [ ]:
save_to_hf_model_name = 'romabob/'+NEW_MODEL
hf_save_token = ''
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
model.push_to_hub(save_to_hf_model_name, token = hf_save_token) # Online saving
tokenizer.push_to_hub(save_to_hf_model_name, token = hf_save_token) # Online saving